# snips-nlu-tutorial for Slot Filling and Intent Prediction
> Main reference: [Snips-NLU document](https://snips-nlu.readthedocs.io/en/latest/) and [Snips-NLU GitHub](https://github.com/snipsco/snips-nlu)

## What are Slot and Intent?

![Snips NLU Processing Pipeline](https://cdn-images-1.medium.com/max/2000/0*Ti7i79hqYC-0lEYM.)

## More Info
+ Paper: [Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces](https://arxiv.org/abs/1805.10190)
+ Product: [Snips](https://snips.ai/)
+ Blog: [An Introduction to Snips NLU, the Open Source Library behind Snips Embedded Voice Platform](https://medium.com/snips-ai/an-introduction-to-snips-nlu-the-open-source-library-behind-snips-embedded-voice-platform-b12b1a60a41a)

## [Installation](https://snips-nlu.readthedocs.io/en/latest/installation.html)

In [ ]:
!pip install snips-nlu
!python -m snips_nlu download en

In [1]:
import io
import json
from snips_nlu.dataset import Dataset
from snips_nlu import SnipsNLUEngine
from snips_nlu.default_configs import CONFIG_EN
from snips_nlu.common.utils import json_string

## [Tutorial](https://snips-nlu.readthedocs.io/en/latest/tutorial.html#tutorial)

In this section, we will build an NLU assistant for home automation tasks. It will be able to understand queries about lights and thermostats. More precisely, our assistant will contain three intents:
+ turnLightOn
+ turnLightOff
+ setTemperature

## Traning Data

In [2]:
dataset_ymal = io.StringIO('''
# turnLightOn intent
---
type: intent
name: turnLightOn
slots:
  - name: room
    entity: room
utterances:
  - Turn on the lights in the [room](kitchen)
  - give me some light in the [room](bathroom) please
  - Can you light up the [room](living room) ?
  - switch the [room](bedroom)'s lights on please

# turnLightOff intent
---
type: intent
name: turnLightOff
slots:
  - name: room
    entity: room
utterances:
  - Turn off the lights in the [room](entrance)
  - turn the [room](bathroom)'s light out please
  - switch off the light the [room](kitchen), will you?
  - Switch the [room](bedroom)'s lights off please

# setTemperature intent
---
type: intent
name: setTemperature
slots:
  - name: room
    entity: room
  - name: roomTemperature
    entity: snips/temperature
utterances:
  - Set the temperature to [roomTemperature](19 degrees) in the [room](bedroom)
  - please set the [room](living room)'s temperature to [roomTemperature](twenty two degrees celsius)
  - I want [roomTemperature](75 degrees fahrenheit) in the [room](bathroom) please
  - Can you increase the temperature to [roomTemperature](22 degrees) ?

# room entity
---
type: entity
name: room
automatically_extensible: no
values:
- bedroom
- [living room, main room, lounge]
- [garden, yard, backyard]''')

Here, we put all the intents and entities in the same file but we could have split them in dedicated files as well.

The setTemperature intent references a roomTemperature slot which relies on the snips/temperature entity. This entity is a builtin entity. It allows to resolve the temperature values properly.

The room entity makes use of synonyms by defining lists like [living room, main room, lounge]. In this case, main room and lounge will point to living room, the first item of the list, which is the reference value.

Besides, this entity is marked as not automatically extensible which means that the NLU will only output values that we have defined and will not try to match other values.

In [15]:
# snips-nlu generate-dataset en dataset.yaml > dataset.json
dataset = Dataset.from_yaml_files("en", [dataset_ymal])

## The Snips NLU Engine

In [4]:
engine = SnipsNLUEngine(config=CONFIG_EN)

## Training the engine

In [5]:
engine.fit(dataset.json)

## Parsing

In [6]:
engine.parse(u"Please give me some lights in the entrance !")

{'input': 'Please give me some lights in the entrance !',
 'intent': {'intentName': 'turnLightOn', 'probability': 0.6521438361130067},
 'slots': [{'range': {'start': 34, 'end': 42},
   'rawValue': 'entrance',
   'value': {'kind': 'Custom', 'value': 'entrance'},
   'entity': 'room',
   'slotName': 'room'}]}

In [7]:
parsing = engine.parse(u"Hey, lights on in the lounge !")
print(json.dumps(parsing, indent=2))

{
  "input": "Hey, lights on in the lounge !",
  "intent": {
    "intentName": "turnLightOn",
    "probability": 0.6426581422388888
  },
  "slots": [
    {
      "range": {
        "start": 22,
        "end": 28
      },
      "rawValue": "lounge",
      "value": {
        "kind": "Custom",
        "value": "living room"
      },
      "entity": "room",
      "slotName": "room"
    }
  ]
}


Notice that the lounge slot value points to living room as defined earlier in the entity synonyms of the dataset.

Now, let’s say the intent is already known and provided by the context of the application, but the slots must still be extracted. A second parsing API allows to extract the slots while providing the intent:

In [8]:
parsing = engine.get_slots(u"Hey, lights on in the lounge !", "turnLightOn")
print(json.dumps(parsing, indent=2))

[
  {
    "range": {
      "start": 22,
      "end": 28
    },
    "rawValue": "lounge",
    "value": {
      "kind": "Custom",
      "value": "living room"
    },
    "entity": "room",
    "slotName": "room"
  }
]


Finally, there is another method that allows to run only the intent classification and get the list of intents along with their score:

In [9]:
intents = engine.get_intents(u"Hey, lights on in the lounge !")
print(json.dumps(intents, indent=2))

[
  {
    "intentName": "turnLightOn",
    "probability": 0.6426581422388888
  },
  {
    "intentName": null,
    "probability": 0.31152377377334123
  },
  {
    "intentName": "turnLightOff",
    "probability": 0.18934883142013287
  },
  {
    "intentName": "setTemperature",
    "probability": 0.14796236731325962
  }
]


### Persisting

As a final step, we will persist the engine into a directory. That may be useful in various contexts, for instance if you want to train on a machine and parse on another one.

You can persist the engine with the following API:

In [13]:
engine.persist("engines")

And load it:

In [14]:
loaded_engine = SnipsNLUEngine.from_path("engines")

loaded_engine.parse(u"Turn lights on in the bathroom please")

{'input': 'Turn lights on in the bathroom please',
 'intent': {'intentName': 'turnLightOn', 'probability': 0.6545729693278636},
 'slots': [{'range': {'start': 22, 'end': 30},
   'rawValue': 'bathroom',
   'value': {'kind': 'Custom', 'value': 'bathroom'},
   'entity': 'room',
   'slotName': 'room'}]}

Alternatively, you can persist/load the engine as a bytearray:

In [16]:
engine_bytes = engine.to_byte_array()
loaded_engine = SnipsNLUEngine.from_byte_array(engine_bytes)